In [1]:
# ------------------------------------------------------------------------------------
# Going through all image-gt pairs and collect all dice scores with gamma + otsu local
# ------------------------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path
from skimage.io import imread
from typing import Union, Tuple

# Current folder as project_root
# Going three layers of folders upwards to src
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
src_dir      = os.path.join(project_root)

# Add the src directory to the Python module search path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# Import functions from src
import importlib
import src.Dice_Score
importlib.reload(src.Dice_Score)

from src.pre_processing import gammacorrection
from src.Complete_Otsu_Global import otsu_threshold_skimage_like
from src.Otsu_Local import local_otsu_fast
from src.Dice_Score import dice_score

# Process single image and its groundtruth
def process_single(img_path: Path, gt_path: Path, s) -> float:
    """ 
    Reads one image and corresponding groundtruth, proccesses, segments and computes dice score for one image.
    """
    # Reads image and reads, binarizes groundtruth
    img = imread(img_path, as_gray=True)
    gt = imread(gt_path, as_gray=True)
    gt  = 1 - (gt == 0)         

    # Gamma correction
    img_gamma = gammacorrection(img, gamma=0.6)
    
    # local Otsu tresholding
    t = local_otsu_fast(img_gamma, radius=s)

    # binarize
    binary = (img_gamma > t).astype(np.uint8)

    # 5. Compute Dice score
    return dice_score(binary.flatten(), gt.flatten())

# data set: NIH3T3
img_dir = Path(project_root) / "data-git" / "NIH3T3" / "img"
gt_dir = Path(project_root) / "data-git" / "NIH3T3" / "gt"

radius_dict = {
    0: 1100,
    1: 500,
    26: 300,
    27: 300,
    28: 300,
    29: 300,
    30: 300,
    31: 300,
    32: 200,
    33: 400,
    37: 300,
    40: 300,
    42: 300,
    44: 200,
    45: 200,
    46: 300,
    47: 600,
    49: 200
}

dice_scores_NIH3T3 = []

for img_file in sorted(img_dir.glob("dna-*.png")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('-')[-1]
    gt_file = gt_dir / f"{idx}.png"
    idx = int(idx)
    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue
    
    r = radius_dict.get(idx)
    if not isinstance(r, int):
     raise ValueError(f"Invalid radius for index {idx}: {r}")

    score = process_single(img_file, gt_file, s=r)
    dice_scores_NIH3T3.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_NIH3T3 = np.array(dice_scores_NIH3T3)           
print("\nDONE ⇢ Mean Dice:", dice_scores_NIH3T3.mean())

np.save('NIH3T3_dice_scores_gamma+local', dice_scores_NIH3T3)



# data set: N2DL-HeLa
img_dir = Path(project_root) / "data-git" / "N2DL-HeLa" / "img"
gt_dir = Path(project_root) / "data-git" / "N2DL-HeLa" / "gt"

radius_dict = {
    13: 300,
    52: 300,
    75: 300,
    79: 300
}

dice_scores_N2DL_HeLa = []

for img_file in sorted(img_dir.glob("t*.tif")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('t')[-1]
    gt_file = gt_dir / f"man_seg{idx}.tif"
    idx = int(idx)

    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue

    r = radius_dict.get(idx)

    score = process_single(img_file, gt_file, s=r)
    dice_scores_N2DL_HeLa.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_N2DL_HeLa = np.array(dice_scores_N2DL_HeLa)           
print("\nDONE ⇢ Mean Dice:", dice_scores_N2DL_HeLa.mean())

np.save('N2DL-HeLa_dice_scores_gamma+local', dice_scores_N2DL_HeLa)


# data set: N2DH-GOWT1
img_dir = Path(project_root) / "data-git" / "N2DH-GOWT1" / "img"
gt_dir = Path(project_root) / "data-git" / "N2DH-GOWT1" / "gt"

radius_dict = {
    1: 1400,
    21: 1600,
    31: 1500,
    39: 1400,
    52: 1800,
    72: 2800
}

dice_scores_N2DH_GOWT1 = []

for img_file in sorted(img_dir.glob("t*.tif")):
    # Extract index to read the corresponding gt
    idx = img_file.stem.split('t')[-1]
    gt_file = gt_dir / f"man_seg{idx}.tif"
    idx = int(idx)

    if not gt_file.exists():
        print(f" ! no Ground-Truth for {img_file.name} !skip …")
        continue
    
    r = radius_dict.get(idx)

    score = process_single(img_file, gt_file, s=r)
    dice_scores_N2DH_GOWT1.append(score)
    print(f"{img_file.name:<20} Dice = {score:.4f}")

# Vector containing all dice scores
dice_scores_N2DH_GOWT1 = np.array(dice_scores_N2DH_GOWT1)           
print("\nDONE ⇢ Mean Dice:", dice_scores_N2DH_GOWT1.mean())

np.save('N2DH-GOWT1_dice_scores_gamma+local', dice_scores_N2DH_GOWT1)

KeyboardInterrupt: 